In [48]:
# typical set up
import pandas as pd
import numpy as np
import json
import glob
import matplotlib.pyplot as plt
%matplotlib inline

pd.options.display.max_rows = 999
pd.options.display.max_columns=100
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.figsize'] = [15, 10]

dataDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/'
jsonDir = '/Users/rachelanderson/Dropbox (Princeton)/Data for Tax Equity Project/JSONFiles/'

In [49]:
with open(jsonDir + 'final_parent_dict.json') as file:
    parent_map = json.load(file)

In [196]:
new_gen_df = pd.read_excel(dataDir + 'eia_860m/march_generator2020.xlsx',header=1)
new_gen_df.columns = new_gen_df.columns.str.lower().str.replace(" ", "_")
new_gen_df.drop(new_gen_df.tail(1).index,inplace=True)

In [199]:
new_gen_df['parent_id']=new_gen_df['entity_name'].str.lower().map(parent_map)

In [201]:
missing_parent = new_gen_df[(new_gen_df['parent_id'].isna())]

In [202]:
new_gen_df['parent_id'].fillna(new_gen_df['plant_name'],inplace=True)

In [203]:
old_gen_df = pd.read_csv(dataDir + 'master_gen_by_owner_2028.csv')
old_gen_df.drop(columns='Unnamed: 0', axis=1,inplace=True)

/Users/rachelanderson/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (16,39,43,77,90) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [205]:
# select the overlapping plant names
missing_parent_ids = old_gen_df[(old_gen_df['plant_name'].isin(missing_parent['plant_name']))].utility_name.str.lower().map(parent_map)

In [209]:
missing_parent_map = dict(zip(missing_parent['plant_name'], missing_parent_ids))

In [210]:
new_gen_df.parent_id = new_gen_df.parent_id.replace(missing_parent_map)

### check up to 2020 stuff

In [217]:
new_gen_df['nameplate_cap'] = new_gen_df['nameplate_capacity_(mw)']

In [223]:
solar=new_gen_df[new_gen_df['technology']=="Solar Photovoltaic"]

In [231]:
solar.groupby('parent_id').nameplate_cap.sum().sort_values(ascending=False).reset_index()

,parent_id,nameplate_cap
0,17650,2359.6
1,56769,2352.6
2,58468,1693.1
3,6452,1602.7
4,61060,1391.8
...,...,...
1499,61440,0.2
1500,13943,0.2
1501,11427,0.1
1502,61332,0.1


In [230]:
decode_parent_map = {
    95: 'nextera',
    5: 'southern current',
    63: 'tenaska capital',
    137: 'exelon',
    56769: 'consolidated edison',
    1442: 'dominion',
    1348: 'cypress creek renewables',
    128: 'southern power co',
    227: 'spower',
    10: '8me',
    320: 'berkshire hathaway',
    3363: 'solar star',
    1694: 'first solar asset management',
    2653: 'sunedison',
    1465: 'nrg',
    363: 'nrg',
    6: '174 power',
    1230: 'vistra',
    566: 'arevon',
    3084: 'canadian solar',
    3403: 'southern power co',
    612: 'duke energy',
    1695: 'first solar project development'
    
    
}